Anticipez les besoins en consommation électrique de bâtiments
=============================================================

![logo-seattle](../reports/figures/logo-seattle.png)


Explication des variables:
[City of seattle](https://data.seattle.gov/dataset/2015-Building-Energy-Benchmarking/h7rm-fz6m)

In [1]:
import os
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster

In [2]:
data = dict()
data_dir = os.path.abspath('../data/raw')
for file in os.listdir(data_dir):
    if file.endswith('.csv'):
        key = file.split('.')[0]
        data[key] = pd.read_csv(os.path.join(data_dir, file))

In [3]:
data = pd.concat(data, sort=False)

In [4]:
data.head()

OSEBuildingID  DataYear    BuildingType  \
2015-building-energy-benchmarking 0              1      2015  NonResidential   
                                  1              2      2015  NonResidential   
                                  2              3      2015  NonResidential   
                                  3              5      2015  NonResidential   
                                  4              8      2015  NonResidential   

                                    PrimaryPropertyType  \
2015-building-energy-benchmarking 0               Hotel   
                                  1               Hotel   
                                  2               Hotel   
                                  3               Hotel   
                                  4               Hotel   

                                              PropertyName  \
2015-building-energy-benchmarking 0   MAYFLOWER PARK HOTEL   
                                  1        PARAMOUNT HOTEL   
                                  2           WESTIN HOTEL   
                                  3              HOTEL MAX   
                                  4  WARWICK SEATTLE HOTEL   

                                    TaxParcelIdentificationNumber  \
2015-building-energy-benchmarking 0                     659000030   
                                  1                     659000220   
                                  2                     659000475   
                                  3                     659000640   
                                  4                     659000970   

                                                                              Location  \
2015-building-energy-benchmarking 0  {'latitude': '47.61219025', 'longitude': '-122...   
                                  1  {'latitude': '47.61310583', 'longitude': '-122...   
                                  2  {'latitude': '47.61334897', 'longitude': '-122...   
                                  3  {'latitude': '47.61421585', 'longitude': '-122...   
                                  4  {'latitude': '47.6137544', 'longitude': '-122....   

                                     CouncilDistrictCode Neighborhood  \
2015-building-energy-benchmarking 0                    7     DOWNTOWN   
                                  1                    7     DOWNTOWN   
                                  2                    7     DOWNTOWN   
                                  3                    7     DOWNTOWN   
                                  4                    7     DOWNTOWN   

                                     YearBuilt  ...  Zip Codes  Address  City  \
2015-building-energy-benchmarking 0       1927  ...    18081.0      NaN   NaN   
                                  1       1996  ...    18081.0      NaN   NaN   
                                  2       1969  ...    18081.0      NaN   NaN   
                                  3       1926  ...    18081.0      NaN   NaN   
                                  4       1980  ...    19576.0      NaN   NaN   

                                     State  ZipCode Latitude Longitude  \
2015-building-energy-benchmarking 0    NaN      NaN      NaN       NaN   
                                  1    NaN      NaN      NaN       NaN   
                                  2    NaN      NaN      NaN       NaN   
                                  3    NaN      NaN      NaN       NaN   
                                  4    NaN      NaN      NaN       NaN   

                                     Comments TotalGHGEmissions  \
2015-building-energy-benchmarking 0       NaN               NaN   
                                  1       NaN               NaN   
                                  2       NaN               NaN   
                                  3       NaN               NaN   
                                  4       NaN               NaN   

                                     GHGEmissionsIntensity  
2015-building-energy-benchmarking 0  

In [5]:
data.describe()

,OSEBuildingID,DataYear,CouncilDistrictCode,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseTypeGFA,...,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes,ZipCode,Latitude,Longitude,Comments,TotalGHGEmissions,GHGEmissionsIntensity
count,6716.000000,6716.00000,6716.000000,6716.000000,6708.000000,6708.000000,6.716000e+03,6716.000000,6.716000e+03,6.560000e+03,...,3338.000000,213.000000,3338.000000,3340.000000,3360.000000,3376.000000,3376.000000,0.0,3367.000000,3367.000000
mean,20867.420488,2015.50268,4.445057,1968.101549,1.071407,4.695736,9.281410e+04,9519.597975,8.329450e+04,7.707936e+04,...,32.380168,1.347418,24.825644,18786.680240,98116.949107,47.624033,-122.334795,NaN,119.723971,1.175916
std,11878.968174,0.50003,2.123749,33.020391,1.593505,5.476218,1.875062e+05,33868.050881,1.733470e+05,1.681617e+05,...,19.907567,0.477272,15.006287,588.604192,18.615205,0.047758,0.027203,NaN,538.832227,1.821452
min,1.000000,2015.00000,1.000000,1900.000000,0.000000,0.000000,1.128500e+04,-3.000000,-5.055000e+04,5.656000e+03,...,1.000000,1.000000,1.000000,17916.000000,98006.000000,47.499170,-122.414250,NaN,-0.800000,-0.020000
25%,19934.000000,2015.00000,3.000000,1948.000000,1.000000,2.000000,2.848700e+04,0.000000,2.727300e+04,2.501600e+04,...,14.000000,1.000000,10.000000,18379.000000,98105.000000,47.599860,-122.350662,NaN,9.495000,0.210000
50%,23064.000000,2016.00000,4.000000,1974.000000,1.000000,4.000000,4.400800e+04,0.000000,4.218500e+04,3.953000e+04,...,33.000000,1.000000,26.000000,18390.000000,98115.000000,47.618675,-122.332495,NaN,33.920000,0.610000
75%,25868.250000,2016.00000,7.000000,1996.000000,1.000000,5.000000,8.943125e+04,0.000000,8.112000e+04,7.417800e+04,...,50.000000,2.000000,38.000000,19576.000000,98122.000000,47.657115,-122.319407,NaN,93.940000,1.370000
max,50226.000000,2016.00000,7.000000,2015.000000,111.000000,99.000000,9.320156e+06,512608.000000,9.320156e+06,9.320156e+06,...,61.000000,2.000000,51.000000,19584.000000,98272.000000,47.733870,-122.220966,NaN,16870.980000,34.090000


In [6]:
for col in data.columns:
    print(col)

OSEBuildingID
DataYear
BuildingType
PrimaryPropertyType
PropertyName
TaxParcelIdentificationNumber
Location
CouncilDistrictCode
Neighborhood
YearBuilt
NumberofBuildings
NumberofFloors
PropertyGFATotal
PropertyGFAParking
PropertyGFABuilding(s)
ListOfAllPropertyUseTypes
LargestPropertyUseType
LargestPropertyUseTypeGFA
SecondLargestPropertyUseType
SecondLargestPropertyUseTypeGFA
ThirdLargestPropertyUseType
ThirdLargestPropertyUseTypeGFA
YearsENERGYSTARCertified
ENERGYSTARScore
SiteEUI(kBtu/sf)
SiteEUIWN(kBtu/sf)
SourceEUI(kBtu/sf)
SourceEUIWN(kBtu/sf)
SiteEnergyUse(kBtu)
SiteEnergyUseWN(kBtu)
SteamUse(kBtu)
Electricity(kWh)
Electricity(kBtu)
NaturalGas(therms)
NaturalGas(kBtu)
OtherFuelUse(kBtu)
GHGEmissions(MetricTonsCO2e)
GHGEmissionsIntensity(kgCO2e/ft2)
DefaultData
Comment
ComplianceStatus
Outlier
2010 Census Tracts
Seattle Police Department Micro Community Policing Plan Areas
City Council Districts
SPD Beats
Zip Codes
Address
City
State
ZipCode
Latitude
Longitude
Comments
TotalGHGEmi

In [7]:
data.head()

OSEBuildingID  DataYear    BuildingType  \
2015-building-energy-benchmarking 0              1      2015  NonResidential   
                                  1              2      2015  NonResidential   
                                  2              3      2015  NonResidential   
                                  3              5      2015  NonResidential   
                                  4              8      2015  NonResidential   

                                    PrimaryPropertyType  \
2015-building-energy-benchmarking 0               Hotel   
                                  1               Hotel   
                                  2               Hotel   
                                  3               Hotel   
                                  4               Hotel   

                                              PropertyName  \
2015-building-energy-benchmarking 0   MAYFLOWER PARK HOTEL   
                                  1        PARAMOUNT HOTEL   
                                  2           WESTIN HOTEL   
                                  3              HOTEL MAX   
                                  4  WARWICK SEATTLE HOTEL   

                                    TaxParcelIdentificationNumber  \
2015-building-energy-benchmarking 0                     659000030   
                                  1                     659000220   
                                  2                     659000475   
                                  3                     659000640   
                                  4                     659000970   

                                                                              Location  \
2015-building-energy-benchmarking 0  {'latitude': '47.61219025', 'longitude': '-122...   
                                  1  {'latitude': '47.61310583', 'longitude': '-122...   
                                  2  {'latitude': '47.61334897', 'longitude': '-122...   
                                  3  {'latitude': '47.61421585', 'longitude': '-122...   
                                  4  {'latitude': '47.6137544', 'longitude': '-122....   

                                     CouncilDistrictCode Neighborhood  \
2015-building-energy-benchmarking 0                    7     DOWNTOWN   
                                  1                    7     DOWNTOWN   
                                  2                    7     DOWNTOWN   
                                  3                    7     DOWNTOWN   
                                  4                    7     DOWNTOWN   

                                     YearBuilt  ...  Zip Codes  Address  City  \
2015-building-energy-benchmarking 0       1927  ...    18081.0      NaN   NaN   
                                  1       1996  ...    18081.0      NaN   NaN   
                                  2       1969  ...    18081.0      NaN   NaN   
                                  3       1926  ...    18081.0      NaN   NaN   
                                  4       1980  ...    19576.0      NaN   NaN   

                                     State  ZipCode Latitude Longitude  \
2015-building-energy-benchmarking 0    NaN      NaN      NaN       NaN   
                                  1    NaN      NaN      NaN       NaN   
                                  2    NaN      NaN      NaN       NaN   
                                  3    NaN      NaN      NaN       NaN   
                                  4    NaN      NaN      NaN       NaN   

                                     Comments TotalGHGEmissions  \
2015-building-energy-benchmarking 0       NaN               NaN   
                                  1       NaN               NaN   
                                  2       NaN               NaN   
                                  3       NaN               NaN   
                                  4       NaN               NaN   

                                     GHGEmissionsIntensity  
2015-building-energy-benchmarking 0  

In [8]:
data['Location']

2015-building-energy-benchmarking  0       {'latitude': '47.61219025', 'longitude': '-122...
                                   1       {'latitude': '47.61310583', 'longitude': '-122...
                                   2       {'latitude': '47.61334897', 'longitude': '-122...
                                   3       {'latitude': '47.61421585', 'longitude': '-122...
                                   4       {'latitude': '47.6137544', 'longitude': '-122....
                                                                 ...                        
2016-building-energy-benchmarking  3371                                                  NaN
                                   3372                                                  NaN
                                   3373                                                  NaN
                                   3374                                                  NaN
                                   3375                               

In [9]:
location = lambda x : x if pd.isna(x) else eval(x)
data['Location'] = data['Location'].apply(location)

In [14]:
data.loc['2016-building-energy-benchmarking'][['Latitude', 'Longitude']]

,Latitude,Longitude
0,47.61220,-122.33799
1,47.61317,-122.33393
2,47.61393,-122.33810
3,47.61412,-122.33664
4,47.61375,-122.34047
...,...,...
3371,47.56722,-122.31154
3372,47.59625,-122.32283
3373,47.63644,-122.35784
3374,47.52832,-122.32431


In [15]:
m = folium.Map(location=(47.6, -122.3),
               tiles='cartodbpositron',
               zoom_start=12)

mc = FastMarkerCluster(data.loc['2016-building-energy-benchmarking'][['Latitude', 'Longitude']].values.tolist())
mc.add_to(m)

m

In [ ]:
def convert_data_to_si_unit(data):
    for col in data.columns:
        if "(kbtu)" in col.lower():
            print(col)
            data[col] = data[col].apply(kbtu_to_kwh)
            col_2 = col.replace("(kBtu)", "(kwh)")
            data.rename(columns={col: col_2}, inplace=True)
            
        elif "(kbtu/sf)" in col.lower():
            print(col)
            data[col] = data[col].apply(kbtu_per_sf_to_kwh_per_sm)
            col_2 = col.replace("(kBtu/sf)", "(kwh/m2)")
            data.rename(columns={col: col_2}, inplace=True)
            
        else:
            pass


In [ ]:
convert_data_to_si_unit(data)

In [ ]:
data.describe()

In [ ]:
data['SiteEUI(kwh/m2)'].describe()

In [ ]:
for col in data.columns:
    print(col)

In [ ]:
data['Electricity(kwh)']

In [ ]:
data['Electricity(kWh)'] / data['Electricity(kwh)']